In [61]:
import os
import sys
import requests
from bs4 import BeautifulSoup as bs
import re

In [62]:
page = requests.post("https://enr-apps.as.cmu.edu/open/SOC/SOCServlet/search", data={"SEMESTER": "F18" ,"MINI": "NO" ,"GRAD_UNDER": "All"\
,"PRG_LOCATION": "All", "DEPT": "All" ,"BEG_TIME": "All"})
page_text = str(page.content)
page_text = page_text.replace(r'\n', '').replace(r'\r', '').replace("   ","")
soup = bs(page_text, "html5lib")

In [63]:
import xlsxwriter
workbook = xlsxwriter.Workbook('/Users/youzhang/documents/cmu/18spring/python/project/demo.xlsx')
sheet = workbook.add_worksheet("class_sheet")
raw_class_sheet = workbook.add_worksheet("raw_class_sheet")

In [65]:
table_list = soup.find_all('table')
row = 0
for item in table_list:
    elements = item.tbody.find_all('tr')
    for element in elements:
        raw_class_sheet.write(row, 0, element.text)
        td_list = element.find_all('td')
        if element.a is not None:
            current_number = element.a.text
            sheet.write(row, 0, current_number)
            course_name = td_list[1].text.strip()
            sheet.write(row, 1, course_name)
            unit = td_list[2].text.strip()
            sheet.write(row, 2, unit)
        section = td_list[3].text.strip()
        sheet.write(row, 3, section)
        days = td_list[5].text.strip()
        sheet.write(row, 4, days)
        start_time = td_list[6].text.strip()
        sheet.write(row, 5, start_time)
        end_time = td_list[7].text.strip()
        sheet.write(row, 6, end_time)
        location = td_list[8].text.strip()
        sheet.write(row, 7, location)
        classroom = td_list[9].text.strip()
        sheet.write(row, 8, classroom)
        instructor = td_list[10].text.strip()
        sheet.write(row, 9, instructor)
        row+=1
        
        


In [66]:
sheet = workbook.add_worksheet("teacher_sheet")
raw_teacher_sheet = workbook.add_worksheet("raw_teacher_sheet")

In [67]:
import json
start = 0
row = 0
raw_row = 0
while start<723:
    try:
        teacher_page = requests.get('http://search.mtvnservices.com/typeahead/suggest/?solrformat=true&rows=5&callback=noCB&q=*%3A*+AND+schoolid_s%3A181&defType=edismax&qf=teacherfirstname_t%5E2000+teacherlastname_t%5E2000+teacherfullname_t%5E2000+autosuggest&bf=pow(total_number_of_ratings_i%2C2.1)&sort=total_number_of_ratings_i+desc&siteName=rmp&&fl=pk_id+teacherfirstname_t+teacherlastname_t+total_number_of_ratings_i+averageratingscore_rf+schoolid_s&fq='+'&start='+str(start))
        teacher_content = str(teacher_page.content)
        teacher_content = teacher_content.replace(r"\n",'')
        teacher_content = teacher_content[teacher_content.find('{'):teacher_content.rfind('}')+1]
        raw_teacher_sheet.write(raw_row, 0, teacher_content)
        raw_row+=1
        data = json.loads(teacher_content)
        teacher_list = data['response']['docs']
        for element in teacher_list:
            last_name = element['teacherlastname_t']
            score = element['averageratingscore_rf']
            first_name = element['teacherfirstname_t']
            sheet.write(row, 0, ','.join([last_name, first_name]))
            sheet.write(row, 1, score)
            row+=1
        if len(teacher_list)<5:
            break
    except:
        print("")
    start+=5
workbook.close()

In [68]:
workbook.close()